<center>
<img src="../../img/ods_stickers.jpg">
## Open Machine Learning Course
<center>
Author: Yury Kashnitsky, Data Scientist at Mail.Ru Group

This material is subject to the terms and conditions of the license [Creative Commons CC BY-NC-SA 4.0](https://creativecommons.org/licenses/by-nc-sa/4.0/). Free use is permitted for any non-comercial purpose with an obligatory indication of the names of the authors and of the source.

## <center>Assignment #6. Part 1
### <center> Beating benchmarks in "Catch Me If You Can: Intruder Detection through Webpage Session Tracking"
    
[Competition](https://www.kaggle.com/c/catch-me-if-you-can-intruder-detection-through-webpage-session-tracking2). The task is to beat "Assignment 6 baseline".

Based on Kaggle kernels:
    - https://www.kaggle.com/kashnitsky/correct-time-aware-cross-validation-scheme/code
    - 

In [98]:
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
from matplotlib import pyplot as plt
import seaborn as sns
import os
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix, hstack
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, TfidfTransformer
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import TimeSeriesSplit, cross_val_score, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from collections import Counter
import pickle
from sklearn.pipeline import Pipeline

Reading original data

In [2]:
PATH_TO_DATA = ('/Users/owner/data/project_alice')
train_df = pd.read_csv(os.path.join(PATH_TO_DATA, 'train_sessions.csv'), index_col='session_id')
test_df = pd.read_csv(os.path.join(PATH_TO_DATA, 'test_sessions.csv'), index_col='session_id')

Separate target feature 

In [3]:
y = train_df['target']

In [4]:
train_df.head()

,site1,time1,site2,time2,site3,time3,site4,time4,site5,time5,...,time6,site7,time7,site8,time8,site9,time9,site10,time10,target
session_id,,,,,,,,,,,,,,,,,,,,,
1,718,2014-02-20 10:02:45,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2,890,2014-02-22 11:19:50,941.0,2014-02-22 11:19:50,3847.0,2014-02-22 11:19:51,941.0,2014-02-22 11:19:51,942.0,2014-02-22 11:19:51,...,2014-02-22 11:19:51,3847.0,2014-02-22 11:19:52,3846.0,2014-02-22 11:19:52,1516.0,2014-02-22 11:20:15,1518.0,2014-02-22 11:20:16,0
3,14769,2013-12-16 16:40:17,39.0,2013-12-16 16:40:18,14768.0,2013-12-16 16:40:19,14769.0,2013-12-16 16:40:19,37.0,2013-12-16 16:40:19,...,2013-12-16 16:40:19,14768.0,2013-12-16 16:40:20,14768.0,2013-12-16 16:40:21,14768.0,2013-12-16 16:40:22,14768.0,2013-12-16 16:40:24,0
4,782,2014-03-28 10:52:12,782.0,2014-03-28 10:52:42,782.0,2014-03-28 10:53:12,782.0,2014-03-28 10:53:42,782.0,2014-03-28 10:54:12,...,2014-03-28 10:54:42,782.0,2014-03-28 10:55:12,782.0,2014-03-28 10:55:42,782.0,2014-03-28 10:56:12,782.0,2014-03-28 10:56:42,0
5,22,2014-02-28 10:53:05,177.0,2014-02-28 10:55:22,175.0,2014-02-28 10:55:22,178.0,2014-02-28 10:55:23,177.0,2014-02-28 10:55:23,...,2014-02-28 10:55:59,175.0,2014-02-28 10:55:59,177.0,2014-02-28 10:55:59,177.0,2014-02-28 10:57:06,178.0,2014-02-28 10:57:11,0


In [5]:
site_cols = [col for col in train_df.columns if 'site' in col]
times = ['time%s' % i for i in range(1, 11)]

In [6]:
train_df[times] = train_df[times].apply(pd.to_datetime)
test_df[times] = test_df[times].apply(pd.to_datetime)
train_df[site_cols].fillna(float(0)).head()

,site1,site2,site3,site4,site5,site6,site7,site8,site9,site10
session_id,,,,,,,,,,
1,718,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,890,941.0,3847.0,941.0,942.0,3846.0,3847.0,3846.0,1516.0,1518.0
3,14769,39.0,14768.0,14769.0,37.0,39.0,14768.0,14768.0,14768.0,14768.0
4,782,782.0,782.0,782.0,782.0,782.0,782.0,782.0,782.0,782.0
5,22,177.0,175.0,178.0,177.0,178.0,175.0,177.0,177.0,178.0


In [7]:
# Concatenate train and test data
full_df = pd.concat([train_df.drop(columns='target', axis = 1),test_df])
idx_train = train_df.shape[0]

In [8]:
full_df[site_cols].fillna(float(0)).head()

,site1,site2,site3,site4,site5,site6,site7,site8,site9,site10
session_id,,,,,,,,,,
1,718,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,890,941.0,3847.0,941.0,942.0,3846.0,3847.0,3846.0,1516.0,1518.0
3,14769,39.0,14768.0,14769.0,37.0,39.0,14768.0,14768.0,14768.0,14768.0
4,782,782.0,782.0,782.0,782.0,782.0,782.0,782.0,782.0,782.0
5,22,177.0,175.0,178.0,177.0,178.0,175.0,177.0,177.0,178.0


In [9]:
#Let's use only numeric sites feature"
full_df['str0']=full_df.apply(lambda x: str( " ".join([str(a) for a in x.values if a != 0])), axis=1)

In [10]:
full_df.str0.head()

session_id
1    718 2014-02-20 10:02:45 nan NaT nan NaT nan Na...
2    890 2014-02-22 11:19:50 941.0 2014-02-22 11:19...
3    14769 2013-12-16 16:40:17 39.0 2013-12-16 16:4...
4    782 2014-03-28 10:52:12 782.0 2014-03-28 10:52...
5    22 2014-02-28 10:53:05 177.0 2014-02-28 10:55:...
Name: str0, dtype: object

In [11]:
def get_auc_score_log_reg(X,y, C =1, ratio = 0.7 ):
    idx_train = int(round(X.shape[0]*ratio))
    #Train logistic regression model
    log_reg = LogisticRegression(C = C, n_jobs = -1)
    log_reg.fit(X[:idx_train], y[:idx_train])
    
    #Make predictions
    y_pred = log_reg.predict_proba(X[idx_train:])[:,1]
    # ROCAUC score
    score = roc_auc_score(y[idx_train:],y_pred)
    return score
    

Build Tf-Idf features based on sites. You can use `ngram_range`=(1, 3) and `max_features`=100000 or more

In [46]:
vcrz = CountVectorizer(ngram_range=(1,3), max_features=100000)
counts_train = vcrz.fit_transform(full_df['str0'][:idx_train].ravel(), y) # ravel() flattens the structure
counts_test = vcrz.fit_transform(full_df['str0'][idx_train:].ravel(), y) # ravel() flattens the structure

In [47]:
tfidf = TfidfTransformer()
X_tr = tfidf.fit_transform(counts_train)
X_test = tfidf.fit_transform(counts_test)


In [60]:
for C in range(3,6):
    print(C,get_auc_score_log_reg(X_tr,y, C = C))

3 0.996881951662
4 0.997417340795
5 0.997724759142


In [12]:
pipeline = Pipeline([('vcrz', TfidfVectorizer(ngram_range=(1,3), max_features=100000)),('tfidf', TfidfTransformer())])
pipeline.fit(full_df.str0[:idx_train].ravel(),y)

X_tr0 = pipeline.transform(full_df.str0[:idx_train].ravel())
X_test0 = pipeline.transform(full_df.str0[idx_train:].ravel())

for C in range(3,6):
    print(C, get_auc_score_log_reg(X_tr0, y, C=C))

3 0.997087932498
4 0.997524510646
5 0.997777998977


In [74]:
def conc(l):
    inv_site_dic = dict()
    for a in l:
        if a != 0:
            inv_site_dic[str(a)] = str(a)
    return inv_site_dic

In [75]:
full_df['str'] = full_df[site_cols].apply(lambda x: "".join(conc(x)), axis = 1)

In [77]:
full_df['str']=full_df['str'].apply(lambda x: x.replace('.', ' '))

In [78]:
full_df['str'].head()

session_id
1                                   718 0nan
2    890 0941 03847 0942 03846 01516 01518 0
3                     14769 039 014768 037 0
4                                      782 0
5                        22 0177 0175 0178 0
Name: str, dtype: object

In [79]:
pipeline = Pipeline([('vectorize',TfidfVectorizer(ngram_range = (1, 3), max_features = 100000)),\
                     ('tfidf',TfidfTransformer())])
pipeline.fit(full_df['str'][:idx_train].ravel(),y)

X_train = pipeline.transform(full_df.str[:idx_train].ravel())
X_test = pipeline.transform(full_df.str[idx_train:].ravel())
for C in range(3,6):
    print(C, get_auc_score_log_reg(X_train,y, C= C))

3 0.946129329815
4 0.946713185028
5 0.94676055832


In [80]:
LR=LogisticRegression(C=5,  n_jobs=-1, random_state=17)
LR.fit(X_tr0, y)
test_pred=LR.predict_proba(X_test0)


In [92]:
X_tr0.shape

(253561, 100000)

Add features based on the session start time: hour, whether it's morning, day or night and so on.

In [82]:
full_df.head(2)

,site1,time1,site2,time2,site3,time3,site4,time4,site5,time5,...,site7,time7,site8,time8,site9,time9,site10,time10,str0,str
session_id,,,,,,,,,,,,,,,,,,,,,
1,718,2014-02-20 10:02:45,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,...,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,718 2014-02-20 10:02:45 nan NaT nan NaT nan Na...,718 0nan
2,890,2014-02-22 11:19:50,941.0,2014-02-22 11:19:50,3847.0,2014-02-22 11:19:51,941.0,2014-02-22 11:19:51,942.0,2014-02-22 11:19:51,...,3847.0,2014-02-22 11:19:52,3846.0,2014-02-22 11:19:52,1516.0,2014-02-22 11:20:15,1518.0,2014-02-22 11:20:16,890 2014-02-22 11:19:50 941.0 2014-02-22 11:19...,890 0941 03847 0942 03846 01516 01518 0


In [93]:
def create_time_feat(df,X_sparse):
    hour = df.time1.apply(lambda ts: ts.hour) 
    morning = ((hour >=6) & (hour <= 11)).astype('int')
    day = ((hour >=12) & (hour <= 18)).astype('int')
    evening = ((hour >=19) & (hour <= 23)).astype('int')
    night = ((hour >= 0) & (hour <= 5)).astype('int')
    X = hstack([X_sparse, morning.values.reshape(-1, 1), 
                day.values.reshape(-1, 1), evening.values.reshape(-1, 1), 
                night.values.reshape(-1, 1)])
    return X

In [95]:
%%time
X_train_new = create_time_feat(full_df[:idx_train], X_tr0)


CPU times: user 1.5 s, sys: 293 ms, total: 1.79 s
Wall time: 1.88 s


In [96]:
X_test_new = create_time_feat(full_df[idx_train:], X_test0)

Scale this features and combine then with Tf-Idf based on sites (you'll need `scipy.sparse.hstack`)

In [ ]:
# You code here

Perform cross-validation with logistic regression.

In [56]:
pipeline = Pipeline([("vectorize", TfidfVectorizer(ngram_range=(1, 3), max_features=100000)), ("tfidf", TfidfTransformer())])
pipeline.fit(full_df['str'][:idx].ravel(),y)

X_train = pipeline.transform(full_df['str'][:idx].ravel())
X_test = pipeline.transform(full_df['str'][idx:].ravel())
for C in range(3,6):
    print(C, get_auc_lr_valid(X_train, y, C=C))

In [100]:
# See discussion of time series split here:
# https://stats.stackexchange.com/questions/14099/using-k-fold-cross-validation-for-time-series-model-selection
time_split = TimeSeriesSplit(n_splits=10)

In [102]:
[(elem[0].shape, elem[1].shape) for elem in time_split.split(X_train)]

[((23051,), (23051,)),
 ((46102,), (23051,)),
 ((69153,), (23051,)),
 ((92204,), (23051,)),
 ((115255,), (23051,)),
 ((138306,), (23051,)),
 ((161357,), (23051,)),
 ((184408,), (23051,)),
 ((207459,), (23051,)),
 ((230510,), (23051,))]

In [ ]:
#Perform time-series cross-validation with LR

In [103]:
%%time
LR=LogisticRegression(C=5,  n_jobs=-1, random_state=17)
cv_scores = cross_val_score(LR, X_train_new, y, cv=time_split, 
                            scoring='roc_auc', n_jobs=1) # hangs with n_jobs > 1, and locally this runs much faster

CPU times: user 1min 2s, sys: 2.99 s, total: 1min 5s
Wall time: 1min 9s


In [104]:
cv_scores, cv_scores.mean()

(array([ 0.98164574,  0.9916601 ,  0.99543836,  0.99519118,  0.99284164,
         0.9972425 ,  0.99849092,  0.99674554,  0.99814092,  0.99867867]),
 0.9946075572152433)

Make prediction for the test set and form a submission file.

In [107]:
LR.fit(X_train_new, y)
test_pred_1 = LR.predict_proba(X_test_new)[:,1]# You code here

In [108]:
def write_to_submission_file(predicted_labels, out_file,
                             target='target', index_label="session_id"):
    predicted_df = pd.DataFrame(predicted_labels,
                                index = np.arange(1, predicted_labels.shape[0] + 1),
                                columns=[target])
    predicted_df.to_csv(out_file, index_label=index_label)


In [111]:
write_to_submission_file(test_pred_1, "/Users/owner/github/kaggle/assignment6_alice_subm_1.csv")

Now tune C parameter (regularization)

In [112]:
c_values=np.logspace(-2,2,10)
logit_grid_search = GridSearchCV(estimator = LR, param_grid= {'C': c_values}, scoring = 'roc_auc', n_jobs= -1,
                                 cv = time_split, verbose = 1)

In [113]:
%%time
logit_grid_search.fit(X_train_new, y)

Fitting 10 folds for each of 10 candidates, totalling 100 fits


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  2.0min
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed: 170.3min finished


CPU times: user 1min 1s, sys: 2.33 s, total: 1min 3s
Wall time: 2h 50min 44s


GridSearchCV(cv=TimeSeriesSplit(max_train_size=None, n_splits=10),
       error_score='raise',
       estimator=LogisticRegression(C=5, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=-1,
          penalty='l2', random_state=17, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'C': array([  1.00000e-02,   2.78256e-02,   7.74264e-02,   2.15443e-01,
         5.99484e-01,   1.66810e+00,   4.64159e+00,   1.29155e+01,
         3.59381e+01,   1.00000e+02])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='roc_auc', verbose=1)

In [115]:
logit_grid_search.best_score_, logit_grid_search.best_params_

(0.99580383130362315, {'C': 100.0})

In [118]:
test_pred_2 = logit_grid_search.predict_proba(X_test_new)[:,1]
write_to_submission_file(test_pred_2, 'assignment6_alice_subm_2.csv')